# 4. Exporting Models to ONNX and Running Inference

This notebook demonstrates how to export the trained/optimized PyTorch models to the ONNX (Open Neural Network Exchange) format. It also shows how to run inference using ONNX Runtime on the exported models.

We will export three specific model versions:
1.  **Baseline FP32 Model**: The original MobileNetV2 model adapted for CIFAR-10.
3.  **Baseline QAT INT8 Model**: The baseline model quantized to INT8 using Quantization-Aware Training.

In [1]:
import os
import torch
import numpy as np
import onnxruntime as ort

from nnopt.model.export import export_model_to_onnx
from nnopt.recipes.mobilenetv2_cifar10 import get_mobilenetv2_cifar10_model
from nnopt.model.prune import remove_pruning_reparameterization
from nnopt.model.const import BASE_MODEL_DIR, DEVICE

# Ensure the logger in export is configured (if not already by its import)
import logging
logger = logging.getLogger("nnopt.model.export")
if not logger.hasHandlers():
    logging.basicConfig(level=logging.INFO,
                        format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
                        handlers=[logging.StreamHandler()])

print(f"PyTorch version: {torch.__version__}")
print(f"ONNX Runtime version: {ort.__version__}")
print(f"Using device: {DEVICE}")

2025-06-11 15:15:17,157 - nnopt.recipes.mobilenetv2_cifar10 - INFO - Using device: cuda, dtype: torch.bfloat16


PyTorch version: 2.7.1+cu126
ONNX Runtime version: 1.22.0
Using device: cuda


## Configuration and Dummy Input

Define the paths for the models and a dummy input tensor required for ONNX export. The dummy input shape should match the expected input of the MobileNetV2 model.

In [2]:
# Define model versions to be exported
baseline_fp32_version = "mobilenetv2_cifar10/fp32/baseline"
pruned_fp32_version = "mobilenetv2_cifar10/fp32/l1_unstruct_prune_0.7"
# The QAT model was saved with a nested version structure in 03_quantization
# Original version used for QAT: "mobilenetv2_cifar10/fp32/baseline"
# Saved QAT version: "mobilenetv2_cifar10/int8/qat_mobilenetv2_cifar10/fp32/baseline"
qat_int8_version = "mobilenetv2_cifar10/int8/qat_mobilenetv2_cifar10/fp32/baseline"


# Directory to save ONNX models
ONNX_EXPORT_DIR = os.path.join(BASE_MODEL_DIR, "onnx_exports")
os.makedirs(ONNX_EXPORT_DIR, exist_ok=True)
print(f"ONNX models will be saved in: {ONNX_EXPORT_DIR}")

# Create a dummy input tensor (batch_size, channels, height, width)
# MobileNetV2 typically expects 224x224 images.
# CIFAR-10 images are 32x32, but the model adapts them or uses a standard input size.
# Using 224x224 as per common MobileNetV2 usage and example in pruning notebook.
dummy_input_shape = (1, 3, 224, 224)
dummy_input = torch.randn(dummy_input_shape, device='cpu') # ONNX export prefers CPU dummy input
print(f"Dummy input shape: {dummy_input.shape}")

# Define dynamic axes for batch size flexibility
dynamic_axes = {'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}

ONNX models will be saved in: /home/pbeuran/repos/nnopt/models/onnx_exports
Dummy input shape: torch.Size([1, 3, 224, 224])


## 1. Baseline FP32 Model Export & Inference

In [3]:
# Load the baseline FP32 model
print(f"Loading baseline FP32 model from version: {baseline_fp32_version}")
baseline_fp32_model, _ = get_mobilenetv2_cifar10_model(
    version=baseline_fp32_version
)
baseline_fp32_model.eval()
baseline_fp32_model.to('cpu') # Move model to CPU for export

# Define ONNX path
onnx_path_baseline_fp32 = os.path.join(ONNX_EXPORT_DIR, "mobilenetv2_cifar10_baseline_fp32.onnx")

# Export to ONNX
print(f"Exporting baseline FP32 model to {onnx_path_baseline_fp32}...")
success_fp32 = export_model_to_onnx(
    model=baseline_fp32_model,
    dummy_input=dummy_input,
    onnx_path=onnx_path_baseline_fp32,
    dynamic_axes=dynamic_axes,
    opset_version=13
)

if success_fp32:
    print("Baseline FP32 model exported successfully.")
    # Run inference with ONNX Runtime
    try:
        ort_session_fp32 = ort.InferenceSession(onnx_path_baseline_fp32, providers=['CPUExecutionProvider'])
        input_name_fp32 = ort_session_fp32.get_inputs()[0].name
        output_name_fp32 = ort_session_fp32.get_outputs()[0].name
        
        ort_inputs_fp32 = {input_name_fp32: dummy_input.cpu().numpy()}
        ort_outputs_fp32 = ort_session_fp32.run([output_name_fp32], ort_inputs_fp32)
        print(f"ONNX Runtime (FP32 Baseline) output shape: {ort_outputs_fp32[0].shape}")
        # print(f"ONNX Runtime (FP32 Baseline) output sample: {ort_outputs_fp32[0][0,:5]}") # Print first 5 logits
    except Exception as e:
        print(f"Error running ONNX Runtime for FP32 baseline model: {e}")
else:
    print("Baseline FP32 model export failed.")

2025-06-11 15:15:17,188 - nnopt.recipes.mobilenetv2_cifar10 - INFO - Loading MobileNetV2 model for CIFAR-10 (architecture and weights) from version: mobilenetv2_cifar10/fp32/baseline at /home/pbeuran/repos/nnopt/models


Loading baseline FP32 model from version: mobilenetv2_cifar10/fp32/baseline


2025-06-11 15:15:17,418 - nnopt.recipes.mobilenetv2_cifar10 - INFO - Successfully loaded model from /home/pbeuran/repos/nnopt/models/mobilenetv2_cifar10/fp32/baseline/model.pt
2025-06-11 15:15:17,420 - nnopt.recipes.mobilenetv2_cifar10 - INFO - Loaded metadata: {'metrics_values': {'val_metrics': {'accuracy': 0.8988, 'avg_loss': 0.2952394811630249, 'samples_per_second': 8822.506935259542, 'avg_time_per_batch': 0.007173827088484393, 'avg_time_per_sample': 0.00011334646799805342, 'params_stats': {'int_weight_params': 0, 'float_weight_params': 2202560, 'float_bias_params': 10, 'bn_param_params': 34112, 'other_float_params': 0, 'total_params': 2236682, 'approx_memory_mb_for_params': 8.532264709472656}}, 'test_metrics': {'accuracy': 0.9004, 'avg_loss': 0.29234390649795533, 'samples_per_second': 8450.564700334979, 'avg_time_per_batch': 0.007537279433337601, 'avg_time_per_sample': 0.00011833528710340034, 'params_stats': {'int_weight_params': 0, 'float_weight_params': 2202560, 'float_bias_param

Exporting baseline FP32 model to /home/pbeuran/repos/nnopt/models/onnx_exports/mobilenetv2_cifar10_baseline_fp32.onnx...


2025-06-11 15:15:17,894 - nnopt.model.export - INFO - Model successfully exported to /home/pbeuran/repos/nnopt/models/onnx_exports/mobilenetv2_cifar10_baseline_fp32.onnx


Baseline FP32 model exported successfully.
ONNX Runtime (FP32 Baseline) output shape: (1, 10)


### Evaluate Baseline FP32 ONNX Model

Now, let's use the `eval_onnx_model` function to evaluate the exported baseline FP32 ONNX model. We'll try this on both CPU and GPU (if available) to compare performance and accuracy.

In [4]:
# Import necessary functions and data
from nnopt.model.eval import eval_onnx_model
from nnopt.recipes.mobilenetv2_cifar10 import get_cifar10_datasets

# Load CIFAR-10 test dataset
# Assuming the data is already downloaded and preprocessed as in other notebooks
# Adjust data_dir if your CIFAR-10 data is located elsewhere
DATA_DIR = os.path.join(os.getcwd(), '..', 'data', 'image', 'cifar10') 
_, test_dataset, _ = get_cifar10_datasets() # We only need test_dataset

if success_fp32: # Only proceed if the ONNX model was exported successfully
    print("\n--- Evaluating Baseline FP32 ONNX Model on CPU ---")
    onnx_metrics_cpu = eval_onnx_model(
        onnx_model_path=onnx_path_baseline_fp32,
        test_dataset=test_dataset,
        batch_size=32, # Adjust as needed
        device="cpu",
        num_warmup_batches=2 # Smaller warmup for quicker testing
    )
    print(f"CPU ONNX Metrics: {onnx_metrics_cpu}")

    if torch.cuda.is_available() and ort.get_device() == 'GPU':
        print("\n--- Evaluating Baseline FP32 ONNX Model on GPU ---")
        onnx_metrics_gpu = eval_onnx_model(
            onnx_model_path=onnx_path_baseline_fp32,
            test_dataset=test_dataset,
            batch_size=32, # Adjust as needed
            device="cuda",
            num_warmup_batches=2
        )
        print(f"GPU ONNX Metrics: {onnx_metrics_gpu}")
    else:
        print("\nSkipping GPU ONNX evaluation as CUDA is not available or ONNX Runtime GPU provider is not set up.")
else:
    print("\nSkipping ONNX model evaluation as the export failed.")

2025-06-11 15:15:19,754 - nnopt.recipes.mobilenetv2_cifar10 - INFO - Loading existing training and validation datasets...
2025-06-11 15:15:21,903 - nnopt.recipes.mobilenetv2_cifar10 - INFO - Loading existing test dataset...
2025-06-11 15:15:22,140 - nnopt.model.eval - INFO - Starting ONNX model evaluation for: /home/pbeuran/repos/nnopt/models/onnx_exports/mobilenetv2_cifar10_baseline_fp32.onnx
2025-06-11 15:15:22,141 - nnopt.model.eval - INFO - Evaluation on PyTorch device: cpu, batch size: 32
2025-06-11 15:15:22,141 - nnopt.model.eval - INFO - Using ONNX Runtime providers: ['CPUExecutionProvider']
2025-06-11 15:15:22,163 - nnopt.model.eval - INFO - ONNX Model Input Name: input, Output Name: output
2025-06-11 15:15:22,164 - nnopt.model.eval - INFO - Starting warmup for 2 batches...



--- Evaluating Baseline FP32 ONNX Model on CPU ---


[ONNX Warmup]: 100%|██████████| 2/2 [00:00<00:00,  3.78it/s]
2025-06-11 15:15:22,805 - nnopt.model.eval - INFO - Warmup complete.
2025-06-11 15:15:22,806 - nnopt.model.eval - INFO - Starting ONNX model evaluation pass...
[ONNX Evaluation]: 100%|██████████| 157/157 [00:21<00:00,  7.27it/s]

ONNX Evaluation Complete: Avg Loss: 0.2994, Accuracy: 0.8970
Throughput: 291.97 samples/sec | Avg Batch Time: 109.08 ms | Avg Sample Time: 3.43 ms
System Stats (PyTorch side): CPU Usage: 81.00% | RAM Usage: 8.5/30.9GB (33.4%)
CPU ONNX Metrics: {'accuracy': 0.897, 'avg_loss': 0.29936650104522705, 'samples_per_second': 291.9695883088593, 'avg_time_per_batch': 0.1090768868854669, 'avg_time_per_sample': 0.003425014248203661}

Skipping GPU ONNX evaluation as CUDA is not available or ONNX Runtime GPU provider is not set up.


## 2. Baseline QAT INT8 Model Export & Inference

For QAT models, PyTorch's `torch.quantization.convert` (typically called at the end of QAT) prepares the model with quantized weights and operations. When loading a saved QAT model, it should already be in this converted state if saved correctly. The `get_mobilenetv2_cifar10_model` with `quantized=True` loads `torchvision.models.quantization.mobilenet_v2` which is suitable for QAT models.

In [9]:
# Load the Baseline QAT INT8 model
print(f"Loading Baseline QAT INT8 model from version: {qat_int8_version}")
# The QAT model is saved after torch.quantization.convert, so it's already an INT8 model.
# We use quantized=True in get_mobilenetv2_cifar10_model to load the correct model architecture
# (e.g., torchvision.models.quantization.mobilenet_v2)
qat_int8_model, _ = get_mobilenetv2_cifar10_model(
    version=qat_int8_version,
    device='cpu' # Ensure model is loaded on CPU for export
)
qat_int8_model.eval()
qat_int8_model.to('cpu') # Quantized models run on CPU. ONNX export also expects CPU model.

# Define ONNX path
onnx_path_qat_int8 = os.path.join(ONNX_EXPORT_DIR, "mobilenetv2_cifar10_qat_int8.onnx")

# Export to ONNX
# Opset version 13+ is generally recommended for better support of quantized operators.
print(f"Exporting Baseline QAT INT8 model to {onnx_path_qat_int8}...")
success_qat_int8 = export_model_to_onnx(
    model=qat_int8_model,
    dummy_input=dummy_input, # Dummy input should be FP32 for QAT model export
    onnx_path=onnx_path_qat_int8,
    dynamic_axes=dynamic_axes,
    opset_version=13 # Use opset 13 or higher for QAT models
)

if success_qat_int8:
    print("Baseline QAT INT8 model exported successfully.")
    # Run inference with ONNX Runtime
    try:
        ort_session_qat_int8 = ort.InferenceSession(onnx_path_qat_int8, providers=['CPUExecutionProvider'])
        input_name_qat_int8 = ort_session_qat_int8.get_inputs()[0].name
        output_name_qat_int8 = ort_session_qat_int8.get_outputs()[0].name
        
        # Input to ONNX Runtime for QAT model is also FP32
        ort_inputs_qat_int8 = {input_name_qat_int8: dummy_input.cpu().numpy()}
        ort_outputs_qat_int8 = ort_session_qat_int8.run([output_name_qat_int8], ort_inputs_qat_int8)
        print(f"ONNX Runtime (QAT INT8) output shape: {ort_outputs_qat_int8[0].shape}")
    except Exception as e:
        print(f"Error running ONNX Runtime for QAT INT8 model: {e}")
else:
    print("Baseline QAT INT8 model export failed.")

2025-06-11 15:17:54,605 - nnopt.recipes.mobilenetv2_cifar10 - INFO - Loading MobileNetV2 model for CIFAR-10 (architecture and weights) from version: mobilenetv2_cifar10/int8/qat_mobilenetv2_cifar10/fp32/baseline at /home/pbeuran/repos/nnopt/models
2025-06-11 15:17:54,658 - nnopt.recipes.mobilenetv2_cifar10 - INFO - Successfully loaded model from /home/pbeuran/repos/nnopt/models/mobilenetv2_cifar10/int8/qat_mobilenetv2_cifar10/fp32/baseline/model.pt
2025-06-11 15:17:54,659 - nnopt.recipes.mobilenetv2_cifar10 - INFO - Loaded metadata: {'metrics_values': {'accuracy': 0.8762, 'avg_loss': 0.36691694159507754, 'samples_per_second': 685.1813385203897, 'avg_time_per_batch': 0.046479861560056304, 'avg_time_per_sample': 0.001459467652985768, 'params_stats': {'int_weight_params': 2202560, 'float_weight_params': 0, 'float_bias_params': 17066, 'bn_param_params': 0, 'other_float_params': 0, 'total_params': 2219626, 'approx_memory_mb_for_params': 2.1656265258789062}}}


Loading Baseline QAT INT8 model from version: mobilenetv2_cifar10/int8/qat_mobilenetv2_cifar10/fp32/baseline


AttributeError: 'ConvReLU2d' object has no attribute '_modules'

In [10]:
qat_int8_version

'mobilenetv2_cifar10/int8/qat_mobilenetv2_cifar10/fp32/baseline'

### Evaluate Baseline QAT INT8 ONNX Model

Finally, we evaluate the exported baseline QAT INT8 ONNX model using `eval_onnx_model`. 
Note: QAT INT8 models are typically run on CPU. While ONNX Runtime supports some quantized operators on GPU via TensorRT or CUDA execution providers, CPU is the more common target for these models. We will primarily test on CPU here.

In [ ]:
# Ensure test_dataset is loaded

if success_qat_int8: # Only proceed if the ONNX model was exported successfully
    print("\n--- Evaluating Baseline QAT INT8 ONNX Model on CPU ---")
    onnx_metrics_qat_int8_cpu = eval_onnx_model(
        onnx_model_path=onnx_path_qat_int8,
        test_dataset=test_dataset,
        batch_size=32, 
        device="cpu", # QAT models are typically evaluated on CPU
        num_warmup_batches=2
    )
    print(f"CPU ONNX Metrics (QAT INT8): {onnx_metrics_qat_int8_cpu}")

    # Optional: Test QAT INT8 on GPU if supported and desired
    # Note: GPU support for INT8 ONNX models can be more complex and might require specific ONNX opset versions
    # or specific GPU capabilities and ONNX Runtime build options.
    if torch.cuda.is_available() and ort.get_device() == 'GPU':
        print("\n--- Evaluating Baseline QAT INT8 ONNX Model on GPU (Experimental) ---")
        try:
            onnx_metrics_qat_int8_gpu = eval_onnx_model(
                onnx_model_path=onnx_path_qat_int8,
                test_dataset=test_dataset,
                batch_size=32, 
                device="cuda",
                num_warmup_batches=2
            )
            print(f"GPU ONNX Metrics (QAT INT8): {onnx_metrics_qat_int8_gpu}")
        except Exception as e:
            print(f"Could not run QAT INT8 ONNX model on GPU: {e}")
            print("This might be due to operator support or other configuration issues.")
    else:
        print("\nSkipping GPU ONNX evaluation for QAT INT8 model as CUDA is not available or ONNX Runtime GPU provider is not set up.")
else:
    print("\nSkipping ONNX model evaluation for QAT INT8 model as the export failed.")

With these steps, the three specified models are exported to ONNX format and tested with ONNX Runtime. These ONNX models can now be used for deployment or further conversion to other formats like TensorRT (for NVIDIA GPUs).

# Analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from nnopt.model.eval import eval_model # For PyTorch model evaluation
from nnopt.recipes.mobilenetv2_cifar10 import get_mobilenetv2_cifar10_model, get_cifar10_datasets # To load models and dataset

# Ensure test_dataset is loaded (it should be from earlier cells, e.g., cell d52e6d98)
# If not, uncomment and run:
# DATA_DIR = os.path.join(os.getcwd(), '..', 'data', 'image', 'cifar10')
# _, _, test_dataset = get_cifar10_datasets(data_dir=DATA_DIR) # Or however you load it

# --- 1. PyTorch FP32 Baseline Model Evaluation (CPU) ---
print("Evaluating PyTorch FP32 Baseline Model on CPU for analysis...")
pytorch_fp32_model, _ = get_mobilenetv2_cifar10_model(
    version=baseline_fp32_version, # Defined in cell f411f025
    device='cpu'
)
pytorch_fp32_model.eval()
pytorch_fp32_metrics_cpu = eval_model(
    model=pytorch_fp32_model,
    test_dataset=test_dataset,
    device="cpu",
    use_amp=False, # No AMP for CPU FP32
    dtype=torch.float32,
    batch_size=32, # Consistent batch size
    num_warmup_batches=2
)
print(f"PyTorch FP32 CPU Metrics: {pytorch_fp32_metrics_cpu}")

# --- 2. PyTorch QAT INT8 Baseline Model Evaluation (CPU) ---
print("\nEvaluating PyTorch QAT INT8 Baseline Model on CPU for analysis...")
# qat_int8_version is defined in cell f411f025
pytorch_qat_int8_model, _ = get_mobilenetv2_cifar10_model(
    version=qat_int8_version,
    device='cpu'
)
pytorch_qat_int8_model.eval()
pytorch_qat_int8_metrics_cpu = eval_model(
    model=pytorch_qat_int8_model,
    test_dataset=test_dataset,
    device="cpu",
    use_amp=False, # QAT models are typically run with FP32 interface, actual ops are INT8
    dtype=torch.float32,
    batch_size=32,
    num_warmup_batches=2
)
print(f"PyTorch QAT INT8 CPU Metrics: {pytorch_qat_int8_metrics_cpu}")

# --- 3. Retrieve ONNX Model Metrics (Assumed to be available from previous cells) ---
# Ensure 'onnx_metrics_cpu' and 'onnx_metrics_qat_int8_cpu' are populated from earlier cells
# These should contain keys like 'accuracy', 'avg_time_per_sample'
print(f"\nUsing pre-calculated ONNX FP32 CPU Metrics: {onnx_metrics_cpu}")
print(f"Using pre-calculated ONNX QAT INT8 CPU Metrics: {onnx_metrics_qat_int8_cpu}")


# --- 4. Model Sizes ---
# PyTorch model sizes (from parameters, does not include quantization overhead directly but reflects param precision)
# For a more direct comparison with ONNX file size, you could save the PyTorch models and get file size.
# Here, we use the parameter-based approximation for PyTorch models.
pytorch_fp32_size_mb = pytorch_fp32_metrics_cpu['params_stats']['total_params'] * pytorch_fp32_metrics_cpu['params_stats']['approx_memory_mb_for_params'] / pytorch_fp32_metrics_cpu['params_stats']['total_params'] if pytorch_fp32_metrics_cpu['params_stats']['total_params'] > 0 else 0
# For QAT INT8, the parameters are still stored in FP32 for training, but effective size is smaller.
# The 'params_stats' from eval_model for a quantized model might still reflect FP32 storage if not careful.
# A better measure for PyTorch quantized model size is to save it and check file size, or estimate based on INT8.
# For simplicity, we'll use the reported size from eval_model, but acknowledge it might be an overestimate for QAT.
# A more accurate way for PyTorch INT8 model size:
torch.save(pytorch_qat_int8_model.state_dict(), "temp_qat_int8_model.pth")
pytorch_qat_int8_size_mb = os.path.getsize("temp_qat_int8_model.pth") / (1024 * 1024)
os.remove("temp_qat_int8_model.pth")
print(f"PyTorch QAT INT8 Model Size (saved state_dict): {pytorch_qat_int8_size_mb:.2f} MB")


# ONNX model file sizes
# Ensure 'onnx_path_baseline_fp32' and 'onnx_path_qat_int8' are defined (cell f411f025 and 5b5fb4cd, 2170bb29)
onnx_fp32_size_mb = os.path.getsize(onnx_path_baseline_fp32) / (1024 * 1024) if os.path.exists(onnx_path_baseline_fp32) else 0
onnx_qat_int8_size_mb = os.path.getsize(onnx_path_qat_int8) / (1024 * 1024) if os.path.exists(onnx_path_qat_int8) else 0
print(f"ONNX FP32 Model Size: {onnx_fp32_size_mb:.2f} MB")
print(f"ONNX QAT INT8 Model Size: {onnx_qat_int8_size_mb:.2f} MB")


# --- 5. Prepare data for plotting ---
model_labels = [
    "PyTorch FP32",
    "PyTorch QAT INT8",
    "ONNX FP32",
    "ONNX QAT INT8"
]

# Using test accuracies
accuracies = [
    pytorch_fp32_metrics_cpu['accuracy'],
    pytorch_qat_int8_metrics_cpu['accuracy'],
    onnx_metrics_cpu['accuracy'] if 'onnx_metrics_cpu' in locals() and onnx_metrics_cpu else 0, # from cell d52e6d98
    onnx_metrics_qat_int8_cpu['accuracy'] if 'onnx_metrics_qat_int8_cpu' in locals() and onnx_metrics_qat_int8_cpu else 0 # from cell 8b57290a
]

# CPU inference time per sample (test set)
cpu_time_per_sample = [
    pytorch_fp32_metrics_cpu['avg_time_per_sample'],
    pytorch_qat_int8_metrics_cpu['avg_time_per_sample'],
    onnx_metrics_cpu['avg_time_per_sample'] if 'onnx_metrics_cpu' in locals() and onnx_metrics_cpu else float('inf'),
    onnx_metrics_qat_int8_cpu['avg_time_per_sample'] if 'onnx_metrics_qat_int8_cpu' in locals() and onnx_metrics_qat_int8_cpu else float('inf')
]

# Model sizes in MB
model_sizes_mb = [
    pytorch_fp32_size_mb,
    pytorch_qat_int8_size_mb, # Using saved state_dict size
    onnx_fp32_size_mb,
    onnx_qat_int8_size_mb
]

print("\nData for plotting:")
print(f"Labels: {model_labels}")
print(f"Accuracies: {accuracies}")
print(f"CPU Time/Sample (s): {cpu_time_per_sample}")
print(f"Model Sizes (MB): {model_sizes_mb}")

# Check if all ONNX metrics were loaded correctly
if not ('onnx_metrics_cpu' in locals() and onnx_metrics_cpu and \
        'onnx_metrics_qat_int8_cpu' in locals() and onnx_metrics_qat_int8_cpu):
    print("\nWARNING: ONNX metrics might not be fully loaded. Plots might be incomplete or show zero/infinity values.")
    print("Please ensure the cells evaluating ONNX models (d52e6d98, 8b57290a) have been run successfully.")


In [ ]:
# Accuracy Bar Plot (Test Set on CPU)
x = np.arange(len(model_labels))
width = 0.5 # Single bar for test accuracy

fig, ax = plt.subplots(figsize=(10, 6))
rects = ax.bar(x, accuracies, width, label='Test Accuracy (CPU)')

ax.set_ylabel('Accuracy')
ax.set_title('Model Test Accuracy Comparison (CPU)')
ax.set_xticks(x)
ax.set_xticklabels(model_labels, rotation=45, ha="right")
ax.legend()
ax.grid(True, linestyle='--', alpha=0.7)
ax.set_ylim(min(accuracies) * 0.9 if min(accuracies) > 0 else 0, max(accuracies) * 1.1 if max(accuracies) > 0 else 1) # Adjust y-lim dynamically

def autolabel(rects_to_label, ax_to_use):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects_to_label:
        height = rect.get_height()
        ax_to_use.annotate(f'{height:.4f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)

autolabel(rects, ax)

fig.tight_layout()
plt.show()

In [ ]:
# CPU Inference Time Comparison Plot (Time per Sample)
fig, ax = plt.subplots(figsize=(10, 6))
# Convert times to milliseconds for better readability if they are very small
cpu_time_per_sample_ms = [t * 1000 for t in cpu_time_per_sample]
rects = ax.bar(x, cpu_time_per_sample_ms, width, label='CPU Time/Sample (ms)')

ax.set_ylabel('CPU Time/Sample (milliseconds)')
ax.set_title('Model Inference Time Comparison (CPU)')
ax.set_xticks(x)
ax.set_xticklabels(model_labels, rotation=45, ha="right")
ax.legend()
ax.grid(True, linestyle='--', alpha=0.7)
# ax.set_yscale('log') # Use log scale if times vary greatly

def autolabel_time(rects_to_label, ax_to_use):
    for rect in rects_to_label:
        height = rect.get_height()
        ax_to_use.annotate(f'{height:.3f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)

autolabel_time(rects, ax)

fig.tight_layout()
plt.show()

In [ ]:
# Model Size Comparison Plot
fig, ax = plt.subplots(figsize=(10, 6))
rects = ax.bar(x, model_sizes_mb, width, label='Model Size (MB)')

ax.set_ylabel('Model Size (MB)')
ax.set_title('Model Size Comparison')
ax.set_xticks(x)
ax.set_xticklabels(model_labels, rotation=45, ha="right")
ax.legend()
ax.grid(True, linestyle='--', alpha=0.7)

def autolabel_size(rects_to_label, ax_to_use):
    for rect in rects_to_label:
        height = rect.get_height()
        ax_to_use.annotate(f'{height:.2f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)

autolabel_size(rects, ax)

fig.tight_layout()
plt.show()